In [1]:
import os
import mlflow
import requests
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


In [2]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')
# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)

In [3]:
# Load the dataset to a dataframe
df = pd.read_csv(_data_filepath)

# Set the target values
y = df['Cover_Type']#.values

# Set the input values
df.drop('Cover_Type', axis=1, inplace=True)
X = df#.values

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [4]:
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore'),
                                        ["Wilderness_Area", "Soil_Type"]),
                                      remainder='passthrough') # pass all the numeric values through the pipeline without any changes.

column_trans

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['Wilderness_Area', 'Soil_Type'])])

In [5]:

pipe = Pipeline(steps=[("column_trans", column_trans),("scaler", StandardScaler(with_mean=False)), ("RandomForestClassifier", RandomForestClassifier())])

pipe

Pipeline(steps=[('column_trans',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Wilderness_Area',
                                                   'Soil_Type'])])),
                ('scaler', StandardScaler(with_mean=False)),
                ('RandomForestClassifier', RandomForestClassifier())])

In [6]:

param_grid =  {'RandomForestClassifier__max_depth': [1,2,3,10], 'RandomForestClassifier__n_estimators': [10,11]}

search = GridSearchCV(pipe, param_grid, n_jobs=2)
search

GridSearchCV(estimator=Pipeline(steps=[('column_trans',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Wilderness_Area',
                                                                          'Soil_Type'])])),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('RandomForestClassifier',
                                        RandomForestClassifier())]),
             n_jobs=2,
             param_grid={'RandomForestClassifier__max_depth': [1, 2, 3, 10],
                         'RandomForestClassifier__n_estimators': [10, 11]})

In [7]:
search.get_params().keys()

dict_keys(['cv', 'error_score', 'estimator__memory', 'estimator__steps', 'estimator__verbose', 'estimator__column_trans', 'estimator__scaler', 'estimator__RandomForestClassifier', 'estimator__column_trans__n_jobs', 'estimator__column_trans__remainder', 'estimator__column_trans__sparse_threshold', 'estimator__column_trans__transformer_weights', 'estimator__column_trans__transformers', 'estimator__column_trans__verbose', 'estimator__column_trans__verbose_feature_names_out', 'estimator__column_trans__onehotencoder', 'estimator__column_trans__onehotencoder__categories', 'estimator__column_trans__onehotencoder__drop', 'estimator__column_trans__onehotencoder__dtype', 'estimator__column_trans__onehotencoder__handle_unknown', 'estimator__column_trans__onehotencoder__max_categories', 'estimator__column_trans__onehotencoder__min_frequency', 'estimator__column_trans__onehotencoder__sparse', 'estimator__column_trans__onehotencoder__sparse_output', 'estimator__scaler__copy', 'estimator__scaler__wit

In [8]:
import mlflow

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

import os
os.environ['MLFLOW_S3_ENDPOINT_URL'] = "http://10.43.101.192:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'supersecret'

# connect to mlflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mlflow_tracking_examples")

mlflow.autolog(log_model_signatures=True, log_input_examples=True)

with mlflow.start_run(run_name="autolog_with_pipeline") as run:
    search.fit(X_train, y_train)

MlflowException: Cannot set a deleted experiment 'mlflow_tracking_examples' as the active experiment. You can restore the experiment, or permanently delete the experiment to create a new one.

In [ ]:
print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

In [9]:
# Supongamos que guardas un pequeño archivo de texto como artifact
with open("test_artifact.txt", "w") as f:
    f.write("Prueba de conexión a MinIO")
mlflow.log_artifact("test_artifact.txt")